<a href="https://colab.research.google.com/github/microprediction/monteprediction_colab_examples/blob/main/monteprediction_entry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sector Monte Carlo Game

Task:

*   Submit one million scenarios for next week's sector returns
*   Each scenario comprises 11 numbers, one for each sector

Judging:

*   Likelihood ... calculated each week
*   Wealth ... accumulates based on weekly likelihood

Loosely speaking you are rewarded based on how many of your samples are close to the ground truth (and in the case of wealth, how many of your competitors are also close - precise details below).

You can enter by:

* Just running this notebook as is (after changing email and name), or
* Modifying this example notebook first.

If you take the latter path, which is preferred, then you can choose to follow the same rough pattern or devise an entirely different method.

Good luck.

We'll send weekly results by email and there'll be a site soon.



In [ ]:
!pip install scikit-learn
!pip install scipy
!pip install --upgrade monteprediction

import yfinance as yf
import pandas as pd

import numpy as np
from datetime import datetime, timedelta
import time
from monteprediction import SPDR_ETFS
from monteprediction.calendarutil import get_last_wednesday
from monteprediction.submission import send_in_chunks

# Factory defaults
num_samples_per_chunk = int(1048576/8)
num_chunks = 8
num_samples = num_chunks*num_samples_per_chunk

## Step 1. Create a dataframe with just over one million hypothetical weekly returns for each sector.   

Do this however you like this is just an example. One column per sector.

In [2]:
# This example uses Quasi-Monte Carlo on the empirical covariance
# There is absolutely no requirement you follow this pattern

from scipy.stats.qmc import MultivariateNormalQMC
from sklearn.covariance import EmpiricalCovariance

# Get historical weekly returns
last_wednesday = get_last_wednesday()
num_weeks = int(52+4*52*np.random.rand())
start_date = last_wednesday - timedelta(weeks=num_weeks)
data = yf.download(SPDR_ETFS, start=start_date, end=last_wednesday, interval="1wk")
weekly_prices = data['Adj Close']
weekly_returns = weekly_prices.pct_change().dropna()

# Use cov estimation to generate samples
cov_matrix = EmpiricalCovariance().fit(weekly_returns).covariance_
qmc_engine = MultivariateNormalQMC(mean=np.zeros(len(SPDR_ETFS)), cov=cov_matrix)
samples = qmc_engine.random(num_samples)
df = pd.DataFrame(columns=SPDR_ETFS, data = samples)
print(df[:3])

# Verify submission
assert len(df.index)==num_samples,f'Expecting exactly {num_samples} samples'
assert list(df.columns)==SPDR_ETFS,'Columns should match SPDR_ETFS in order'


[*********************100%%**********************]  11 of 11 completed


        XLB       XLC       XLE       XLF       XLI       XLK       XLP  \
0  0.006407  0.024214 -0.041571  0.009535  0.029295  0.035585  0.017758   
1 -0.014000 -0.013554  0.013286 -0.015097 -0.016030  0.012215  0.017820   
2 -0.034061 -0.015787 -0.049576 -0.010769 -0.034243 -0.013276 -0.022992   

       XLRE       XLU       XLV       XLY  
0  0.035765  0.006146  0.013871  0.021571  
1 -0.003764  0.021896  0.002853  0.007079  
2 -0.075759 -0.046329 -0.031928 -0.016339  


## Step 2. Submit the dataframe

In [3]:
YOUR_EMAIL = 'monteprediction_entry@monteprediction.com'  # Be sure to change this
YOUR_NAME  = 'Destroyer of worlds'    # The handle to be used on the leaderboard
send_in_chunks(df, num_chunks=num_chunks, email=YOUR_EMAIL, name=YOUR_NAME)

Chunk 0 of 8 sent successfully.
Chunk 1 of 8 sent successfully.
Chunk 2 of 8 sent successfully.
Chunk 3 of 8 sent successfully.
Chunk 4 of 8 sent successfully.
Chunk 5 of 8 sent successfully.
Chunk 6 of 8 sent successfully.
Chunk 7 of 8 sent successfully.


## Step 3. (Later) verify the submission
(Usually not necessary ... you'll get an email if it failed)

In [5]:
from monteprediction.verification import get_verification_status
import time
time.sleep(10*60)
print(get_verification_status(email=YOUR_EMAIL))

verified


# Explaining the game ...
Here's how the reward system works assuming you are participant $i$. Your samples $\{x_{ik}\}_{k=0}^{n-1}$, where each $x_{ik}$ for $k=0,\dots,2^{20}-1$ is an 11-vector, are used to imply an unnormalized prediction density for $z \in \mathbf{R}^{11}$ as:

$$\rho_i(z) = \frac{1}{n} \sum_{k=0}^{n-1} \exp(-a \|x_{ik}-z \|_2) $$

where $a$ is a system parameter set at approximately $a=300$.

Let us suppose you have an initial wealth $W_i$. A system parameter $b_i=0.1$ is the fraction of your total wealth you deploy. You are considered to invest $\Omega_i = b_i W_i$ and similarly for other participants yielding a total investment of $\Omega = \sum_i \Omega_i$. This pot will be split when the truth $z$ is revealed.

To this end your 'mass' is $Q_i(z) = \Omega_i \rho_i(z)$ represents loosely how many of your samples are close to $z$ weighted by your wealth, and the total mass near $z$ supplied by all participants is $Q(z) = \sum_i Q_i(z)$. Your payout is your propotional share of total investment, namely $\Omega \frac{Q_i(z)}{Q(z)}$. Your net profit is $\delta_i(z) = \Omega \frac{Q_i(z)}{Q(z)} - \Omega_i$.  *italicized text*

It should be apparent that $Q$ plays the role of an unnormalized market probability (i.e. risk-neutral density) and further, that a participant with perfect knowledge of the true density $P$ will at worst break even against any opponents' play, subject only to the ability to approximate $P$ with a collection of Monte Carlo paths in this fashion.

Because this entry has been included in the mix, and is not particularly clever, there is a subsidy for participation for anyone taking even a moment to reflect statistically upon the problem (for instance, by applying shrinkage to the covariance estimation or fixing the 1-margins).


In [ ]:
# The 'score" is your density
#    distances = np.linalg.norm(samples - z, axis=1)
#    score = np.sum(np.exp(-h * distances))

from monteprediction.truth import get_most_recent_truth
from monteprediction.scoring import compute_score
z = get_most_recent_truth()
score = compute_score(samples=df.values,z=z)
print(f"Total Score: {score}")


[*********************100%%**********************]  11 of 11 completed


Total Score: 0.06990740503342224


### Some suggestions

Ask GPT!

*   It will send you to LedoitWolf or ShrunkCovariance from [sklearn.covariance](https://scikit-learn.org/stable/modules/covariance.html)
*   Feel free to use covariance estimation methods from [precise](https://github.com/microprediction/precise/tree/main/precise/skaters/covariance)
*   Python libraries like scipy, statsmodels, or copulas can be useful.
*  Lots of copula articles (e.g. [here](https://medium.com/@crisvelasquez/stock-scenario-modelling-tail-risk-dependence-with-copulas-in-python-25591885366f)).
* Geneative models (e.g. see [this paper](https://cs230.stanford.edu/projects_fall_2019/reports/26259829.pdf))



## Leaderboard

Go to [www.monteprediction.com](www.monteprediction.com)